In [454]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [487]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import re
import string
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split

from dataclasses import dataclass

from train_rnn import train_rnn_multiclass

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torchutils as tu
from torchmetrics.classification import Accuracy

In [456]:
# # Лучше всего установить такую же версию
# print(gensim.__version__)

In [457]:
import pandas as pd

# Замените 'path_to_file.jsonl' на путь к вашему файлу
file_path = '/Users/valeriaalesnikova/Desktop/bootcamp/nlp_project-1/models/restaurants_reviews.jsonl'

# Чтение файла и загрузка данных в список
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Преобразование списка словарей в DataFrame
df = pd.DataFrame(data)
reviews = df['text'].tolist()
# preprocessed = [data_preprocessing(review) for review in reviews]

In [458]:
def data_preprocessing(text: str) -> str:
    """preprocessing string: lowercase, removing html-tags, punctuation and stopwords

    Args:
        text (str): input string for preprocessing

    Returns:
        str: preprocessed string
    """    

    text = text.lower()
    text = re.sub('<.*?>', '', text) # html tags
    text = ''.join([c for c in text if c not in string.punctuation])# Remove punctuation
    text = [word for word in text.split() if word not in stop_words] 
    text = ' '.join(text)
    return text

df['cleaned_text'] = df['text']

In [459]:
df.head(15)

,review_id,general,food,interior,service,text,cleaned_text
0,0,0,10,10,10,Вытянули меня сегодня в город и раз уж была в ...,Вытянули меня сегодня в город и раз уж была в ...
1,1,0,9,10,9,проводили корпоратив на 60 чел. в этот - уже т...,проводили корпоратив на 60 чел. в этот - уже т...
2,2,0,9,10,10,Был в Гостях с женой один раз и еще раз с жено...,Был в Гостях с женой один раз и еще раз с жено...
3,3,0,-,5,10,Бар понравился на первый взгляд . Интерьер к ...,Бар понравился на первый взгляд . Интерьер к ...
4,4,0,7,10,10,В « Bel Canto » мы отмечали юбилей моего отца ...,В « Bel Canto » мы отмечали юбилей моего отца ...
5,5,0,10,10,10,"Здравствуйте , уважаемые женихи и невесты . Г...","Здравствуйте , уважаемые женихи и невесты . Г..."
6,6,0,10,10,10,"Вкусная кухня , уютная остановка , хорошее обс...","Вкусная кухня , уютная остановка , хорошее обс..."
7,7,0,10,10,10,"Вкусная кухня , интересная обстановка , отличн...","Вкусная кухня , интересная обстановка , отличн..."
8,8,0,1,-,1,8 июня заказали 2 пиццы - Бергамо и Юнет . Пи...,8 июня заказали 2 пиццы - Бергамо и Юнет . Пи...
9,9,0,-,5,7,Началась жаркая пора в нашем летнем расписании...,Началась жаркая пора в нашем летнем расписании...


In [460]:
corpus = [word for text in df['cleaned_text'] for word in text.split()]
count_words = Counter(corpus)

sorted_words = count_words.most_common()


In [461]:
def get_words_by_freq(sorted_words: list, n: int = 10) -> list:
    return list(filter(lambda x: x[1] > n, sorted_words))

In [462]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [463]:
reviews_int = []
for text in df['cleaned_text']:

    r = [vocab_to_int[word] for word in text.split() if vocab_to_int.get(word)]
    reviews_int.append(r)

In [464]:
df['general'] = df['general'].apply(lambda x: 1 if x == 'positive' else 0)

In [465]:
review_len = [len(x) for x in reviews_int]
df['Review len'] = review_len
df.head()

,review_id,general,food,interior,service,text,cleaned_text,Review len
0,0,0,10,10,10,Вытянули меня сегодня в город и раз уж была в ...,Вытянули меня сегодня в город и раз уж была в ...,330
1,1,0,9,10,9,проводили корпоратив на 60 чел. в этот - уже т...,проводили корпоратив на 60 чел. в этот - уже т...,158
2,2,0,9,10,10,Был в Гостях с женой один раз и еще раз с жено...,Был в Гостях с женой один раз и еще раз с жено...,103
3,3,0,-,5,10,Бар понравился на первый взгляд . Интерьер к ...,Бар понравился на первый взгляд . Интерьер к ...,157
4,4,0,7,10,10,В « Bel Canto » мы отмечали юбилей моего отца ...,В « Bel Canto » мы отмечали юбилей моего отца ...,354


In [466]:
def padding(review_int: list, seq_len: int) -> np.array:
    """Make left-sided padding for input list of tokens

    Args:
        review_int (list): input list of tokens
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros

    Returns:
        np.array: padded sequences
    """    
    features = np.zeros((len(reviews_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)
            
    return features

In [467]:
def preprocess_single_string(input_string: str, seq_len: int, vocab_to_int: dict = vocab_to_int) -> list:
    """Function for all preprocessing steps on a single string

    Args:
        input_string (str): input single string for preprocessing
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros
        vocab_to_int (dict, optional): word corpus {'word' : int index}. Defaults to vocab_to_int.

    Returns:
        list: preprocessed string
    """    

    preprocessed_string = data_preprocessing(input_string)
    result_list = []
    for word in preprocessed_string.split():
        try: 
            result_list.append(vocab_to_int[word])
        except KeyError as e:
            print(f'{e}: not in dictionary!')
    result_padded = padding([result_list], seq_len)[0]

    return torch.tensor(result_padded)

In [468]:
SEQ_LEN = 32
features = padding(reviews_int, SEQ_LEN)
print(features[3, :])

[ 3608   199     6   250   332     2   129    30 46982  1147    18     5
 17205     1    14   933  2632     1    89  3881     1  1587     2   468
  3073    28   334   950  1100     8  6393     1]


In [469]:
X_train, X_valid, y_train, y_valid = train_test_split(features, df['general'].to_numpy(), test_size=0.2, random_state=1)

In [470]:

train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))


BATCH_SIZE = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)

In [471]:
VOCAB_SIZE = len(vocab_to_int)+1 

In [472]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

In [473]:
device='cpu'

In [474]:
@dataclass
class ConfigRNN:
    vocab_size: int
    device: str
    n_layers: int
    embedding_dim: int
    hidden_size: int
    seq_len: int
    bidirectional: bool or int

In [475]:
# # Конфигурация модели (пример)
# class ConfigRNN:
#     def __init__(self, vocab_size, device, n_layers, embedding_dim, hidden_size, seq_len, bidirectional):
#         self.vocab_size = vocab_size
#         self.device = device
#         self.n_layers = n_layers
#         self.embedding_dim = embedding_dim
#         self.hidden_size = hidden_size
#         self.seq_len = seq_len
#         self.bidirectional = bidirectional

# # Пример словаря и длины последовательности
# vocab_to_int = {'отвратительно': 0, 'плохо': 1, 'средне': 2, 'хорошо': 3, 'отлично': 5}  # пример словаря
# SEQ_LEN = 10

In [476]:
net_config = ConfigRNN(
    vocab_size=len(vocab_to_int) + 1,
    device="cpu",
    n_layers=2,
    embedding_dim=16,
    hidden_size=32,
    seq_len=SEQ_LEN,
    bidirectional=False,
)
net_config

ConfigRNN(vocab_size=282129, device='cpu', n_layers=2, embedding_dim=16, hidden_size=32, seq_len=32, bidirectional=False)

In [477]:
class RNNNet(nn.Module):
    """
    vocab_size: int, размер словаря (аргумент embedding-слоя)
    emb_size:   int, размер вектора для описания каждого элемента последовательности
    hidden_dim: int, размер вектора скрытого состояния, default 0
    batch_size: int, размер batch
    """

    def __init__(self, rnn_conf=net_config) -> None:
        super().__init__()
        self.rnn_conf = rnn_conf
        self.seq_len = rnn_conf.seq_len
        self.emb_size = rnn_conf.embedding_dim
        self.hidden_dim = rnn_conf.hidden_size
        self.n_layers = rnn_conf.n_layers
        self.vocab_size = rnn_conf.vocab_size
        self.bidirectional = bool(rnn_conf.bidirectional)

        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.rnn_cell = nn.RNN(
            input_size=self.emb_size,
            hidden_size=self.hidden_dim,
            batch_first=True,
            bidirectional=self.bidirectional,
            num_layers=self.n_layers,
        )
        self.bidirect_factor = 2 if self.bidirectional == 1 else 1
        self.linear = nn.Sequential(
            nn.Linear(self.hidden_dim * self.seq_len * self.bidirect_factor, 16),
            nn.Tanh(),
            nn.Linear(16, 1)
        )

    def model_description(self):
        direction = "bidirect" if self.bidirectional else "onedirect"
        return f"rnn_{direction}_{self.n_layers}"

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x.to(self.rnn_conf.device))
        output, _ = self.rnn_cell(x)  # Забираем hidden states со всех промежуточных состояний, второй выход отправляем в _
        output = output.contiguous().view(output.size(0), -1)
        out = self.linear(output)
        return out


model_rnn = RNNNet(net_config)
tu.get_model_summary(model_rnn, sample_x.to(net_config.device))

Layer                  Kernel         Output       Params          FLOPs
0_embedding         [16, 282129]   [32, 32, 16]   4,514,064        1,024
1_rnn_cell                     -   [32, 32, 32]       3,712   31,784,960
2_linear.Linear_0     [1024, 16]       [32, 16]      16,400    1,048,064
3_linear.Tanh_1                -       [32, 16]           0        2,560
4_linear.Linear_2        [16, 1]        [32, 1]          17          992
Total params: 4,534,193
Trainable params: 4,534,193
Non-trainable params: 0
Total FLOPs: 32,837,600 / 32.84 MFLOPs
------------------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.38
Params size (MB): 17.30
Estimated Total Size (MB): 17.69


In [482]:
import torch.optim as optim
from torch.optim import Adam

In [483]:
# Инициализация функции потерь, оптимизатора и метрики
criterion = nn.CrossEntropyLoss()
optimizer_rnn = Adam(model_rnn.parameters(), lr=0.001)
metric = Accuracy(task='multiclass', num_classes=5).to(net_config.device)

In [488]:
# цикл обучения и валидации 4,30 минуты

train_losses_rnn, val_losses_rnn, train_metric_rnn, val_metric_rnn, rnn_time = train_rnn_multiclass(
    epochs=5,
    model=model_rnn,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer_rnn,
    rnn_conf=net_config,
    criterion=criterion,
    metric=metric,
)

Epoch 1
train_loss : 0.0000 val_loss : 0.0000
train_accuracy : 1.00 val_accuracy : 1.00
Epoch 2
train_loss : 0.0000 val_loss : 0.0000
train_accuracy : 1.00 val_accuracy : 1.00
Epoch 3
train_loss : 0.0000 val_loss : 0.0000
train_accuracy : 1.00 val_accuracy : 1.00
Epoch 4
train_loss : 0.0000 val_loss : 0.0000
train_accuracy : 1.00 val_accuracy : 1.00
Epoch 5
train_loss : 0.0000 val_loss : 0.0000
train_accuracy : 1.00 val_accuracy : 1.00
